# Decision Tree

In [31]:
# TODO: Write group name, group members, matriculation numbers...
# Group Name: None, group members: None, matriculation number: 220201767

In [32]:
# Following implementation is inspired by: http://gabrielelanaro.github.io/blog/2016/03/03/decision-trees.html
import numpy as np

In [33]:
# Training dataset
# x1 atmospheric pressure
x1 = ['high', 'high', 'low', 'low', 'low', 'high']  
# x2 is weather type
x2 = ['partly cloudy', 'sunny', 'sunny', 'cloudy', 'cloudy', 'cloudy']  
X = np.array([x1, x2]).T
y = np.array([False, False, True, False, False, True]) # rain (True) and no-rain (False)

In [34]:
X # features

array([['high', 'partly cloudy'],
       ['high', 'sunny'],
       ['low', 'sunny'],
       ['low', 'cloudy'],
       ['low', 'cloudy'],
       ['high', 'cloudy']], dtype='<U13')

In [35]:
y # labels

array([False, False,  True, False, False,  True])

In [36]:
# Splitting a set
# Input is an array of feature observations and output is a dictionary with "unique feature value" as key and indices as value
def partition(a):
    return {c: (a==c).nonzero()[0] for c in np.unique(a)}

# Picking which attribute to split
# Calculate entropy of a list
def entropy(s):
    res = 0
    val, counts = np.unique(s, return_counts=True)
    freqs = counts.astype('float')/len(s)
    for p in freqs:
        if p != 0.0:
            res -= p * np.log2(p)
    return res

# Calculate decrease in impurity (information gains)
# 
def mutual_information(y, x):
    
    # Calculate entropy of observation classes
    res = entropy(y)

    # We partition x, according to attribute values x_i
    val, counts = np.unique(x, return_counts=True)
    freqs = counts.astype('float')/len(x)

    # We calculate a weighted average of the entropy and subtract it from parent entropy
    for p, v in zip(freqs, val):
        res -= p * entropy(y[x == v])

    return res

# Checks for pureness of elements in a list
def is_pure(s):
    return len(set(s)) == 1

# Helper function to print decision tree
def print_tree(d, depth = 0):
    for key, value in d.items():
        for i in range(depth):
                print(' ', end='')
        if type(value) is dict:
            print(key, end=':\n')
            print_tree(value, depth + 1)
        else:
            print(key, end=': ')
            print(value)
            
# Get the most common element of an array
def most_common(a):
    (values,counts) = np.unique(a,return_counts=True)
    ind=np.argmax(counts)
    return values[ind]

# Recursive split of observations
def recursive_split(x, y):
    
    # If set to be split is pure or empty, return it as leaf
    if is_pure(y) or len(y) == 0:
        return most_common(y)

    # Calculate decrease in impurity (information gain) and split attribute with maximum gain
    gain = np.array([mutual_information(y, x_attr) for x_attr in x.T])
    selected_attr = np.argmax(gain)

    # Sufficiently pure, return it as leaf
    if np.all(gain < 1e-6):
        return most_common(y)

    # Split using the selected attribute
    sets = partition(x[:, selected_attr])

    # Perform recursive splits and collect results
    res = {}
    for key, value in sets.items():
        y_subset = y.take(value, axis=0)
        x_subset = x.take(value, axis=0)
        res["x_%d = %s" % (selected_attr, key)] = recursive_split(x_subset, y_subset)

    return res

# Perform algorithm on the example dataset to create a decision tree
d = recursive_split(X, y)
print_tree(d)

x_1 = cloudy:
 x_0 = high: True
 x_0 = low: False
x_1 = partly cloudy: False
x_1 = sunny:
 x_0 = high: False
 x_0 = low: True


In [37]:
# New dataset (which shall be classified by the above generated decision tree)
x1 = ['high', 'low', 'low', 'high', 'low', 'high', 'high', 'low', 'low', 'high', 'low', 'low']
x2 = ['sunny', 'sunny', 'cloudy', 'cloudy', 'partly cloudy', 'cloudy', 'partly cloudy', 'cloudy', 'sunny', 'cloudy', 'cloudy', 'partly cloudy']
X = np.array([x1, x2]).T
y = np.array([False, True, True, False, False, True, False, True, True, False, True, True]) # ground-truth of classification

## Solution

In [38]:
def predict_rains(d, X):
    predictions = []
    
    for s in X:
        for k, v in d.items():
            if s[1] in k:
                if isinstance(v, dict):
                    for q, w in v.items():
                        if s[0] in q:
                            if w == True:
                                predictions.append("Rain")
                            else:
                                predictions.append("No Rain")
                            
                else:
                    predictions.append("No Rain")
                
                break
                
    return predictions

In [39]:
predict_y = predict_rains(d, X)

In [40]:
X = np.concatenate((X, np.array([predict_y]).T), axis=1)

In [41]:
X

array([['high', 'sunny', 'No Rain'],
       ['low', 'sunny', 'Rain'],
       ['low', 'cloudy', 'No Rain'],
       ['high', 'cloudy', 'Rain'],
       ['low', 'partly cloudy', 'No Rain'],
       ['high', 'cloudy', 'Rain'],
       ['high', 'partly cloudy', 'No Rain'],
       ['low', 'cloudy', 'No Rain'],
       ['low', 'sunny', 'Rain'],
       ['high', 'cloudy', 'Rain'],
       ['low', 'cloudy', 'No Rain'],
       ['low', 'partly cloudy', 'No Rain']], dtype='<U13')

## Rain

In [46]:
for s in X:
    if s[2] == "Rain":
        print(s)

['low' 'sunny' 'Rain']
['high' 'cloudy' 'Rain']
['high' 'cloudy' 'Rain']
['low' 'sunny' 'Rain']
['high' 'cloudy' 'Rain']


## No Rain

In [47]:
for s in X:
    if s[2] == "No Rain":
        print(s)

['high' 'sunny' 'No Rain']
['low' 'cloudy' 'No Rain']
['low' 'partly cloudy' 'No Rain']
['high' 'partly cloudy' 'No Rain']
['low' 'cloudy' 'No Rain']
['low' 'cloudy' 'No Rain']
['low' 'partly cloudy' 'No Rain']


### Explain the decision tree data structure stored in the variable d. Which python data types are used in which combination? 

In [44]:
print(f"d type: {type(d)}")
print(f"d length: {len(d)}", end="\n\n")

for k, v in d.items():
    print(f"Key type: {type(k)}   -   Value type: {type(v)}")

d type: <class 'dict'>
d length: 3

Key type: <class 'str'>   -   Value type: <class 'dict'>
Key type: <class 'str'>   -   Value type: <class 'numpy.bool_'>
Key type: <class 'str'>   -   Value type: <class 'dict'>


In [45]:
# The structure of decision tree:
# x2 or weather type is picked as root node

# Type of variable d is dictionary of length 3

# First item in d (dictionary) is of type string and dictionary as key and value respectively, 
# the sub-dictionary in first tiem is of type string and bool as key and value respectively.

# Second item is of type string and bool as key and value respectively.

# Third item is of type string and dictionary, again the sub-dictionary here contains items 
# which has the type of string and bool as key and value.